In [0]:
%scala
val storageAccount = "amiranda" //nombre de la cuenta de almacenamiento
val container ="amiranda" //nombre de contenedor
val sasKey = "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T08:34:00Z&st=2024-12-03T00:34:00Z&spr=https&sig=vTb4LTQPSPIQUaQYNhHVj%2FM3Wqx0U2DCel3CKY6Ve1k%3D" //llave de conexion

val conf = "fs.azure.sas." + container + "." + storageAccount + ".blob.core.windows.net"

dbutils.fs.mount(source="wasbs://" + container + "@" + storageAccount + ".blob.core.windows.net", mountPoint="/mnt/" + container, extraConfigs=Map(conf -> sasKey))



storageAccount: String = amiranda
container: String = amiranda
sasKey: String = sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T08:34:00Z&st=2024-12-03T00:34:00Z&spr=https&sig=vTb4LTQPSPIQUaQYNhHVj%2FM3Wqx0U2DCel3CKY6Ve1k%3D
conf: String = fs.azure.sas.amiranda.amiranda.blob.core.windows.net
res2: Boolean = true

In [0]:
#%fs ls /mnt/amiranda

In [0]:
#ingestar archivo producto.parquet
producto = (spark.read.parquet("/mnt/amiranda/proyectofinal/capaplata/Producto.parquet"))
producto.createOrReplaceTempView("s_producto")

In [0]:
#ingestar archivo categoria.parquet
categoria = (spark.read.parquet("/mnt/amiranda/proyectofinal/capaplata/Categoria.parquet"))
categoria.createOrReplaceTempView("s_categoria")

In [0]:
#ingestar archivo subcategoria.parquet
subcategoria = (spark.read.parquet("/mnt/amiranda/proyectofinal/capaplata/SubCategoria.parquet"))
subcategoria.createOrReplaceTempView("s_subcategoria")

In [0]:
%sql
--eliminar tabla en caso exista
drop table if exists Fact_Product;

--crear tabla
create temp view Fact_Product
as
select P.Cod_Producto, P.Nombre as Producto, S.Nombre as SubCategoria, C.Nombre as Categoria
from s_producto P
join s_subcategoria S on P.Cod_SubCategoria = S.Cod_SubCategoria
join s_categoria C on S.Cod_Categoria = C.Cod_Categoria

In [0]:
#resultado se lleva a cada tipo
dfFactProducto = spark.table("Fact_Product")
dfFactProducto.write.mode("overwrite").parquet("/mnt/amiranda/proyectofinal/capaoro/FactProducto.parquet")

In [0]:
#elminar acceso directo
dbutils.fs.unmount("/mnt/amiranda")

/mnt/amiranda has been unmounted.


True